In [ ]:

"""
=============================================================================
BALANCEAMENTO TOGETHER COM VALIDAÇÃO DE CAMINHOS
=============================================================================
Aplica oversampling + augmentation leve no dataset TOGETHER pré-processado.

ESTRATÉGIA:
- Oversampling: Duplica até a classe maior
- Augmentation: LEVE (p=0.3) - conservador
- Preserva todas as imagens originais
- TEST sem modificação (apenas cópia)
- Validação completa de diretórios

CAMINHOS FIXOS:
- INPUT:  ../data/processed/TOGETHER
- OUTPUT: ../data/augmented/TOGETHER
=============================================================================
"""

import os
import cv2
import numpy as np
from tqdm import tqdm
import albumentations as A
from collections import Counter
import warnings
import shutil

warnings.filterwarnings('ignore')

In [ ]:

# =============================================================================
# CONFIGURAÇÕES
# =============================================================================

BASE_PATH = r'../data/processed/TOGETHER'
OUTPUT_PATH = r'../data/augmented/TOGETHER'

# Configurações de Augmentation (LEVE - conservador)
AUGMENTATION_PROBABILITY = 0.3  # 30% chance (leve)
ROTATION_LIMIT = 10
BRIGHTNESS_LIMIT = 0.15
CONTRAST_LIMIT = 0.15
PIXEL_DROPOUT_PROB = 0.005

# Controle
VERBOSE = True


In [25]:
# =============================================================================
# VALIDAÇÃO DE CAMINHOS
# =============================================================================

def validate_and_create_paths():
    """
    Valida diretório de entrada e cria diretório de saída.
    
    Returns:
        tuple: (input_exists: bool, error_message: str)
    """
    print("\n" + "="*80)
    print("VALIDAÇÃO DE CAMINHOS")
    print("="*80)
    
    # Verificar INPUT
    print(f"\n🔍 Verificando INPUT:")
    print(f"   Caminho: {BASE_PATH}")
    
    if not os.path.exists(BASE_PATH):
        error_msg = f"❌ ERRO: Diretório de entrada não encontrado!\n   {BASE_PATH}"
        return False, error_msg
    
    if not os.path.isdir(BASE_PATH):
        error_msg = f"❌ ERRO: Caminho existe mas não é um diretório!\n   {BASE_PATH}"
        return False, error_msg
    
    # Verificar estrutura do dataset
    train_path = os.path.join(BASE_PATH, 'train')
    test_path = os.path.join(BASE_PATH, 'test')
    
    if not os.path.exists(train_path):
        error_msg = f"❌ ERRO: Pasta 'train' não encontrada!\n   {train_path}"
        return False, error_msg
    
    if not os.path.exists(test_path):
        error_msg = f"❌ ERRO: Pasta 'test' não encontrada!\n   {test_path}"
        return False, error_msg
    
    print(f"   ✅ Diretório existe")
    print(f"   ✅ Estrutura válida (train + test)")
    
    # Contar arquivos
    total_train = sum([len([f for f in os.listdir(os.path.join(train_path, d)) 
                            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                       for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))])
    
    total_test = sum([len([f for f in os.listdir(os.path.join(test_path, d)) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                      for d in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, d))])
    
    print(f"   📊 Imagens encontradas:")
    print(f"      • Train: {total_train:,} imagens")
    print(f"      • Test:  {total_test:,} imagens")
    
    # Criar OUTPUT
    print(f"\n📁 Criando OUTPUT:")
    print(f"   Caminho: {OUTPUT_PATH}")
    
    try:
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        print(f"   ✅ Diretório criado/verificado")
    except Exception as e:
        error_msg = f"❌ ERRO: Não foi possível criar diretório de saída!\n   {e}"
        return False, error_msg
    
    print("\n" + "="*80)
    print("✅ VALIDAÇÃO CONCLUÍDA COM SUCESSO")
    print("="*80)
    
    return True, None


In [26]:
# =============================================================================
# PIPELINE DE AUGMENTATION LEVE
# =============================================================================

def create_augmentation_pipeline():
    """
    Augmentation LEVE e CONSERVADOR para balanceamento offline.
    """
    return A.Compose([
        A.OneOf([
            A.HorizontalFlip(p=1.0),
            A.Rotate(limit=ROTATION_LIMIT, p=1.0),
            A.ShiftScaleRotate(
                shift_limit=0.05,
                scale_limit=0.05,
                rotate_limit=ROTATION_LIMIT,
                p=1.0
            ),
            A.RandomBrightnessContrast(
                brightness_limit=BRIGHTNESS_LIMIT,
                contrast_limit=CONTRAST_LIMIT,
                p=1.0
            ),
            A.GaussNoise(var_limit=(5.0, 20.0), p=1.0),
            A.PixelDropout(dropout_prob=PIXEL_DROPOUT_PROB, p=1.0),
            A.ImageCompression(quality_lower=90, quality_upper=100, p=1.0),
            A.Blur(blur_limit=3, p=1.0),
        ], p=AUGMENTATION_PROBABILITY),
    ])




In [27]:
# =============================================================================
# FUNÇÕES DE ANÁLISE
# =============================================================================

def analyze_dataset_distribution(dataset_path):
    """
    Analisa distribuição de classes no dataset.
    """
    distribution = {}
    
    for emotion_folder in os.listdir(dataset_path):
        emotion_path = os.path.join(dataset_path, emotion_folder)
        
        if not os.path.isdir(emotion_path):
            continue
        
        image_files = [
            f for f in os.listdir(emotion_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
        ]
        
        distribution[emotion_folder] = len(image_files)
    
    return distribution


def calculate_oversampling_strategy(distribution):
    """
    Calcula estratégia de oversampling até classe maior.
    """
    target_samples = max(distribution.values())
    strategy = {}
    
    for emotion, count in distribution.items():
        if count < target_samples:
            copies_needed = target_samples // count
            remainder = target_samples % count
            
            strategy[emotion] = {
                'original_count': count,
                'target_count': target_samples,
                'full_copies': copies_needed,
                'partial_samples': remainder,
                'total_new': target_samples - count
            }
        else:
            strategy[emotion] = {
                'original_count': count,
                'target_count': count,
                'full_copies': 1,
                'partial_samples': 0,
                'total_new': 0
            }
    
    return strategy, target_samples


def print_balancing_report(dataset_name, distribution, strategy, target_samples):
    """
    Imprime relatório de balanceamento.
    """
    print("\n" + "="*80)
    print(f"RELATÓRIO DE BALANCEAMENTO - {dataset_name}")
    print("="*80)
    
    print(f"\n📊 Distribuição ORIGINAL (Train):")
    total_original = sum(distribution.values())
    for emotion, count in sorted(distribution.items()):
        percentage = (count / total_original) * 100
        print(f"  • {emotion:15s}: {count:5d} imagens ({percentage:5.2f}%)")
    
    print(f"\n🎯 Target por classe: {target_samples} imagens")
    
    total_new = sum(s['total_new'] for s in strategy.values())
    total_final = total_original + total_new
    
    print(f"\n📈 Mudanças por classe:")
    for emotion, info in sorted(strategy.items()):
        if info['total_new'] > 0:
            print(f"  • {emotion:15s}: {info['original_count']:5d} → {info['target_count']:5d} "
                  f"(+{info['total_new']:5d} novas)")
        else:
            print(f"  • {emotion:15s}: {info['original_count']:5d} (sem mudança)")
    
    print(f"\n📦 Totais (Train):")
    print(f"  • Imagens originais: {total_original:,}")
    print(f"  • Imagens a criar:   {total_new:,}")
    print(f"  • Total final:       {total_final:,}")
    print(f"  • Aumento:           {(total_new/total_original)*100:.1f}%")
    print("="*80)

In [ ]:
# =============================================================================
# PROCESSAMENTO
# =============================================================================


def apply_augmentation_to_image(image, transform):
    """
    Aplica augmentation a uma imagem.
    """
    try:
        augmented = transform(image=image)
        return augmented['image']
    except Exception as e:
        if VERBOSE:
            print(f"⚠️ Erro no augmentation: {e}")
        return image


def balance_dataset(strategy, target_samples, transform):
    """
    Balanceia dataset TOGETHER com oversampling + augmentation.
    TRAIN: balanceamento completo
    TEST: apenas cópia (sem modificação)
    """
    print(f"\n{'-'*80}")
    print("PROCESSANDO TOGETHER")
    print(f"{'-'*80}")
    
    stats = {
        'images_copied': 0,
        'images_created': 0,
        'augmentations_applied': 0,
        'errors': 0
    }
    
    # Processar cada split
    for split_name in ['train', 'test']:
        input_split_path = os.path.join(BASE_PATH, split_name)
        output_split_path = os.path.join(OUTPUT_PATH, split_name)
        
        print(f"\n📂 Processando split: {split_name}")
        
        # ============================================================
        # TEST: apenas copia, SEM balanceamento/augmentation
        # ============================================================
        if split_name == 'test':
            print("  ℹ️  TEST: Copiando imagens originais SEM modificação")
            
            for emotion in strategy.keys():
                emotion_input_path = os.path.join(input_split_path, emotion)
                emotion_output_path = os.path.join(output_split_path, emotion)
                
                if not os.path.exists(emotion_input_path):
                    continue
                
                os.makedirs(emotion_output_path, exist_ok=True)
                
                image_files = [
                    f for f in os.listdir(emotion_input_path)
                    if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
                ]
                
                for img_file in image_files:
                    src = os.path.join(emotion_input_path, img_file)
                    dst = os.path.join(emotion_output_path, img_file)
                    
                    try:
                        shutil.copy2(src, dst)
                        stats['images_copied'] += 1
                    except Exception as e:
                        if VERBOSE:
                            print(f"    ✗ Erro ao copiar {img_file}: {e}")
                        stats['errors'] += 1
                
                print(f"  • {emotion:15s}: {len(image_files)} imagens copiadas")
            
            continue
        
        # ============================================================
        # TRAIN: balanceamento + augmentation
        # ============================================================
        print("  ℹ️  TRAIN: Aplicando balanceamento + augmentation")
        
        for emotion, info in strategy.items():
            emotion_input_path = os.path.join(input_split_path, emotion)
            emotion_output_path = os.path.join(output_split_path, emotion)
            
            if not os.path.exists(emotion_input_path):
                continue
            
            os.makedirs(emotion_output_path, exist_ok=True)
            
            image_files = [
                f for f in os.listdir(emotion_input_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
            ]
            
            if not image_files:
                continue
            
            print(f"  • {emotion:15s}: {info['original_count']} → {info['target_count']}")
            
            # 1. COPIAR ORIGINAIS
            for img_file in tqdm(image_files, desc=f"    Copiando {emotion}", leave=False):
                src = os.path.join(emotion_input_path, img_file)
                base_name = os.path.splitext(img_file)[0]
                ext = os.path.splitext(img_file)[1]
                dst = os.path.join(emotion_output_path, f"original_{base_name}{ext}")
                
                try:
                    shutil.copy2(src, dst)
                    stats['images_copied'] += 1
                except Exception as e:
                    if VERBOSE:
                        print(f"    ✗ Erro ao copiar {img_file}: {e}")
                    stats['errors'] += 1
            
            # 2. GERAR AUMENTADAS
            if info['total_new'] > 0:
                augmentations_per_image = info['total_new'] // len(image_files)
                remainder = info['total_new'] % len(image_files)
                
                for idx, img_file in enumerate(tqdm(image_files, 
                                                    desc=f"    Aumentando {emotion}", 
                                                    leave=False)):
                    src_path = os.path.join(emotion_input_path, img_file)
                    
                    try:
                        img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
                        
                        if img is None:
                            stats['errors'] += 1
                            continue
                        
                        num_augmentations = augmentations_per_image
                        if idx < remainder:
                            num_augmentations += 1
                        
                        for aug_idx in range(num_augmentations):
                            augmented_img = apply_augmentation_to_image(img, transform)
                            
                            base_name = os.path.splitext(img_file)[0]
                            ext = os.path.splitext(img_file)[1]
                            aug_name = f"aug{aug_idx:03d}_{base_name}{ext}"
                            aug_path = os.path.join(emotion_output_path, aug_name)
                            
                            cv2.imwrite(aug_path, augmented_img)
                            stats['images_created'] += 1
                            stats['augmentations_applied'] += 1
                    
                    except Exception as e:
                        if VERBOSE:
                            print(f"    ✗ Erro ao processar {img_file}: {e}")
                        stats['errors'] += 1
    
    return stats

In [ ]:
# =============================================================================
# FUNÇÃO PRINCIPAL
# =============================================================================

def main():
    """
    Função principal - processa TOGETHER.
    """
    print("="*80)
    print("BALANCEAMENTO TOGETHER - VERSÃO CORRIGIDA")
    print("="*80)
    print(f"INPUT:  {BASE_PATH}")
    print(f"OUTPUT: {OUTPUT_PATH}")
    print(f"Augmentation: OneOf com p={AUGMENTATION_PROBABILITY} (LEVE)")
    print("="*80)
    
    # 1. VALIDAR CAMINHOS
    is_valid, error_msg = validate_and_create_paths()
    
    if not is_valid:
        print(f"\n{error_msg}")
        print("\n❌ PROCESSAMENTO ABORTADO - Corrija os caminhos!")
        return
    
    # 2. CRIAR PIPELINE
    print("\n🔧 Criando pipeline de augmentation...")
    transform = create_augmentation_pipeline()
    print("✅ Pipeline criado!")
    
    # 3. ANALISAR TRAIN
    train_path = os.path.join(BASE_PATH, 'train')
    distribution = analyze_dataset_distribution(train_path)
    
    if not distribution:
        print(f"\n❌ Nenhuma classe encontrada em {train_path}")
        return
    
    # 4. CALCULAR ESTRATÉGIA
    strategy, target_samples = calculate_oversampling_strategy(distribution)
    print_balancing_report("TOGETHER",distribution, strategy, target_samples)
    
    # 5. BALANCEAR
    stats = balance_dataset(strategy, target_samples, transform)
    
    # 6. RELATÓRIO FINAL
    print("\n" + "="*80)
    print("PROCESSAMENTO CONCLUÍDO!")
    print("="*80)
    
    print(f"\n📊 Estatísticas Finais:")
    print(f"  • Imagens copiadas: {stats['images_copied']:,}")
    print(f"  • Imagens criadas:  {stats['images_created']:,}")
    print(f"  • Augmentations:    {stats['augmentations_applied']:,}")
    print(f"  • Total final:      {stats['images_copied'] + stats['images_created']:,}")
    
    if stats['errors'] > 0:
        print(f"  ⚠️ Erros: {stats['errors']}")
    
    print(f"\n✅ Dataset balanceado salvo em:")
    print(f"   {OUTPUT_PATH}")
    print("="*80)

In [30]:
# =============================================================================
# PONTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠️ Processamento interrompido pelo usuário.")
    except Exception as e:
        print(f"\n\n❌ ERRO FATAL: {type(e).__name__}")
        print(f"Mensagem: {e}")
        
        import traceback
        print("\nTraceback completo:")
        traceback.print_exc()

BALANCEAMENTO RAF-DB - VERSÃO CORRIGIDA
INPUT:  ../data/processed/RAF-DB
OUTPUT: ../data/augmented/RAF-DB
Augmentation: OneOf com p=0.3 (LEVE)

VALIDAÇÃO DE CAMINHOS

🔍 Verificando INPUT:
   Caminho: ../data/processed/RAF-DB
   ✅ Diretório existe
   ✅ Estrutura válida (train + test)
   📊 Imagens encontradas:
      • Train: 12,271 imagens
      • Test:  3,068 imagens

📁 Criando OUTPUT:
   Caminho: ../data/augmented/RAF-DB
   ✅ Diretório criado/verificado

✅ VALIDAÇÃO CONCLUÍDA COM SUCESSO

🔧 Criando pipeline de augmentation...
✅ Pipeline criado!

RELATÓRIO DE BALANCEAMENTO - RAF-DB

📊 Distribuição ORIGINAL (Train):
  • Felicidade     :  4772 imagens (38.89%)
  • Medo           :   281 imagens ( 2.29%)
  • Neutro         :  2524 imagens (20.57%)
  • Nojo           :   717 imagens ( 5.84%)
  • Raiva          :   705 imagens ( 5.75%)
  • Surpresa       :  1290 imagens (10.51%)
  • Tristeza       :  1982 imagens (16.15%)

🎯 Target por classe: 4772 imagens

📈 Mudanças por classe:
  • Felicid

    Copiando Tristeza:   0%|          | 0/1982 [00:00<?, ?it/s]

  • Raiva          : 705 → 4772


  • Neutro         : 2524 → 4772


  • Surpresa       : 1290 → 4772


  • Felicidade     : 4772 → 4772


  • Medo           : 281 → 4772


  • Nojo           : 717 → 4772



📂 Processando split: test
  ℹ️  TEST: Copiando imagens originais SEM modificação
  • Tristeza       : 478 imagens copiadas
  • Raiva          : 162 imagens copiadas
  • Neutro         : 680 imagens copiadas
  • Surpresa       : 329 imagens copiadas
  • Felicidade     : 1185 imagens copiadas
  • Medo           : 74 imagens copiadas
  • Nojo           : 160 imagens copiadas

PROCESSAMENTO CONCLUÍDO!

📊 Estatísticas Finais:
  • Imagens copiadas: 15,339
  • Imagens criadas:  21,133
  • Augmentations:    21,133
  • Total final:      36,472

✅ Dataset balanceado salvo em:
   ../data/augmented/RAF-DB
